# Importing Any Necessary Packages 

In [4]:
import json 
from pprint import pprint

import pandas as pd
import matplotlib.pyplot as plt
import graphviz

import numpy as np
from pandas.io.json import json_normalize

import math

In [5]:
pd.set_option('display.max_columns', None)

# Extraction of Data 
The raw data set had each of the line as a json, but the entire file was not in json format. Thus we needed to extract the json and convert it into a dataframe. 

In [6]:
def extractDataInfo(filename):
    data = []
    attributes = []
    returnValues = []
    with open(filename) as f:
        for line in f:
            dataLine = json.loads(line)
            data.append(dataLine)
            for key in dataLine.keys():
                attributes.append(key)
        uniqueAttributes = set(attributes)
        returnValues.append(data)
        returnValues.append(uniqueAttributes)
        return returnValues

In [7]:
businessInfo = extractDataInfo('../dataset/business.json') #businessInfo is a list

In [8]:
businessDf = pd.DataFrame.from_dict(businessInfo[0])

In [9]:
originalDf = pd.DataFrame.from_dict(businessInfo[0]) #the original dataset 

# Basic Summary of Raw Business Data

In [10]:
print(businessInfo[0][0])

{'business_id': 'YDf95gJZaq05wvo7hTQbbQ', 'name': 'Richmond Town Square', 'neighborhood': '', 'address': '691 Richmond Rd', 'city': 'Richmond Heights', 'state': 'OH', 'postal_code': '44143', 'latitude': 41.5417162, 'longitude': -81.4931165, 'stars': 2.0, 'review_count': 17, 'is_open': 1, 'attributes': {'RestaurantsPriceRange2': 2, 'BusinessParking': {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, 'BikeParking': True, 'WheelchairAccessible': True}, 'categories': ['Shopping', 'Shopping Centers'], 'hours': {'Monday': '10:00-21:00', 'Tuesday': '10:00-21:00', 'Friday': '10:00-21:00', 'Wednesday': '10:00-21:00', 'Thursday': '10:00-21:00', 'Sunday': '11:00-18:00', 'Saturday': '10:00-21:00'}}


In [11]:
businessDf.shape

(156639, 15)

In [12]:
businessDf.dtypes

address          object
attributes       object
business_id      object
categories       object
city             object
hours            object
is_open           int64
latitude        float64
longitude       float64
name             object
neighborhood     object
postal_code      object
review_count      int64
stars           float64
state            object
dtype: object

A brief description of what the variable contains (i.e. "The Tuition column contains information on the annual cost of tuition in $USD"): 
* "business_id":string, 22 character unique string business id
* "name":string, the business's name
* "neighborhood": string, the neighborhood's name
* "address": string, the full address of the business
* "city": string, the city
* "State": string, 2 character state code, if applicable
* "postal code": string, the postal code
* "latitude": float, latitude
* "longitude": float, longitude
* "stars": float, star rating, rounded to half-stars
* "review_count": interger, number of reviews
* "is_open": integer, 0 or 1 for closed or open, respectively
* "attributes": object, business attributes to values. note: some attribute values might be objects
* "categories": an array of strings of business categories
* "hours":an object of key day to value hours, hours are using a 24hr clock
* More description at https://www.yelp.com/dataset/documentation/json

In [13]:
businessDf.describe()

is_open       latitude      longitude   review_count  \
count  156639.000000  156638.000000  156638.000000  156639.000000   
mean        0.844375      38.585033     -92.856485      30.238159   
std         0.362501       5.399871      26.557741      96.486631   
min         0.000000     -36.086009    -142.466650       3.000000   
25%         1.000000      33.627161    -112.138207       4.000000   
50%         1.000000      36.142381     -89.523198       9.000000   
75%         1.000000      43.596845     -79.668760      23.000000   
max         1.000000      89.999314     115.086769    6979.000000   

               stars  
count  156639.000000  
mean        3.647154  
std         0.977640  
min         1.000000  
25%         3.000000  
50%         3.500000  
75%         4.500000  
max         5.000000

# Expanding DataFrame: Unnesting 
A lot of our data is nested within each dataframe, particularly, the 'attributes', 'hours', and 'categories' columns

### Part I: Unnesting Categories  
We needed to unnest the categories column so we could figure out what type of busineses they were. In particular, we were interested in restaurants. 

In [14]:
def findAllCategories():
    allCategories =[]
    for categoryList in businessDf['categories']:
        if(type(categoryList) != str):  
            for category in categoryList:
                allCategories.append(category)
        else:
            allCategories.append(categoryList)
    return allCategories

In [15]:
def findUniqueCategories(allCategoriesList):
    return set(allCategoriesList)

In [16]:
def categoriesStats(allCategoriesList):
    print('Length of all categories is:')
    print(len(allCategoriesList))
    print('Length of all unique categories is:')
    print(len(findUniqueCategories(allCategoriesList)))
    allCategoriesList.remove('Restaurants')
    allCategoriesSeries = pd.Series(allCategoriesList)
    print(allCategoriesSeries.value_counts())
    
    return allCategoriesSeries.value_counts()

There were 590290 unique labels that would describe a restaurant. 

In [17]:
originalAllCategories = findAllCategories()
len(originalAllCategories)

590290

* statsInfo is essentially the frequency table that shows how many times the label appeared. 
* We created statsInfo because after some manual exploration of the data, we found out that certain labels described a business in a repetitive manner and that those labels could be grouped together. 
* For example, a data could have the labels "American", "Pizza", "Restaurants" and "Food". It was clear that upon inspection that this data point was a restaurant, which is all that we needed to know. 
* Thus, statsInfo was created to give us an intuition on how to group similar or synonymous labels together. 

In [18]:
statsInfo = categoriesStats(originalAllCategories)

Length of all categories is:
590290
Length of all unique categories is:
1240
Restaurants                  51612
Shopping                     24595
Food                         23014
Beauty & Spas                15139
Home Services                13202
Health & Medical             12033
Nightlife                    11364
Bars                          9868
Automotive                    9476
Local Services                9343
Event Planning & Services     8038
Active Life                   7427
Fashion                       6299
Sandwiches                    5864
Fast Food                     5792
American (Traditional)        5737
Pizza                         5652
Coffee & Tea                  5565
Hair Salons                   5395
Hotels & Travel               5188
Arts & Entertainment          5054
Home & Garden                 4584
Auto Repair                   4480
Italian                       4411
Burgers                       4236
Doctors                       4124
Breakfast & B

In [19]:
def getTopCategories(topNum, frequencyTable):
    statsInfoDict = frequencyTable.to_dict()
    statsInfoList = sorted(statsInfoDict.items(), key=lambda x: x[1], reverse=True)
    
    topCategories =[]
    for i in range(topNum):
        topCategories.append(statsInfoList[i][0])
    
    return topCategories

In [20]:
def reGroup(df, categoryName):
    existList = []
    for categoryArr in df:
        exist = False
        if(type(categoryArr) != str):  
            for category in categoryArr:
                if (category == categoryName):
                    exist = True
        existList.append(exist)
    
    return existList 

In [21]:
def replaceCategory(df, columnName, replacement):
    categoriesDf = df[columnName]
    restaurantsExistList = reGroup(categoriesDf, replacement)
    restaurantsExistSeries = pd.Series(restaurantsExistList)
    categoriesDf.loc[restaurantsExistSeries] = replacement
    print('# of ' + replacement + " made:")
    print(len(categoriesDf.loc[categoriesDf == replacement]))

* We took the top 50 labels that were used to describe a data point the most. 
* We decided that these were the most "general" and "dominant" labels and that they could quickly replace redundant labels. 
* We considred them "general" in the sense that the top 50 labels seemed to be pretty broad in its description. For example, it would not be as specific as "DUI Schools" in its descriptions. 
* We considered them as the "dominant" labels because they were the ones that appeared the most. 

In [22]:
topCategoriesList = getTopCategories(50, statsInfo)
topCategoriesList

['Restaurants',
 'Shopping',
 'Food',
 'Beauty & Spas',
 'Home Services',
 'Health & Medical',
 'Nightlife',
 'Bars',
 'Automotive',
 'Local Services',
 'Event Planning & Services',
 'Active Life',
 'Fashion',
 'Sandwiches',
 'Fast Food',
 'American (Traditional)',
 'Pizza',
 'Coffee & Tea',
 'Hair Salons',
 'Hotels & Travel',
 'Arts & Entertainment',
 'Home & Garden',
 'Auto Repair',
 'Italian',
 'Burgers',
 'Doctors',
 'Breakfast & Brunch',
 'Mexican',
 'Nail Salons',
 'Professional Services',
 'American (New)',
 'Chinese',
 'Real Estate',
 'Specialty Food',
 'Fitness & Instruction',
 'Pets',
 'Grocery',
 'Bakeries',
 'Cafes',
 'Hair Removal',
 'Dentists',
 'Hotels',
 'Desserts',
 'Skin Care',
 "Women's Clothing",
 'Education',
 'Japanese',
 'Ice Cream & Frozen Yogurt',
 'Pet Services',
 'Day Spas']

In [23]:
def convertedCategories(df, columnName, dataType):
    isStringList =[]
    dataFrame = df[columnName]
    for observation in dataFrame:
        isString = False
        if(type(observation) == dataType):  
            isString = True
        isStringList.append(isString)
    
    isStringSeries = pd.Series(isStringList)
    print("# converted...")
    converted =len(dataFrame.loc[isStringSeries])
    print(converted)
    print("# unconverted..")
    print(len(df) - converted)

### Regrouping 'Categories' Labels 
We used topCategoriesList to group or redefine the labels of the "categories" column for a given business using the following steps: 
* We iterated through the topCategoriesList in order of the label that appeared the most to the least 
* We took that label and iterated through the entire 'categories' column 
* If the label existed to describe that observation, we replaced the list of categories with that label. 
* For example, if an observation had ["Food", "Italian", "American", and "Restaurants"] we saw that that observation had "Restaurants" in it. While "Food" is in the topCategoriesList, because the "Restaurants" label was the label that appeared most frequently, we replaced the observation in the 'categories' column to 'Restaurants'instead of 'Food'. 

In [24]:
for topCategory in topCategoriesList:
    replaceCategory(businessDf, 'categories', topCategory)
    convertedCategories(businessDf, 'categories', str)
    print("--------------- conversion ended -----------------")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# of Restaurants made:
51613
# converted...
51613
# unconverted..
105026
--------------- conversion ended -----------------
# of Shopping made:
24261
# converted...
75874
# unconverted..
80765
--------------- conversion ended -----------------
# of Food made:
11538
# converted...
87412
# unconverted..
69227
--------------- conversion ended -----------------
# of Beauty & Spas made:
13139
# converted...
100551
# unconverted..
56088
--------------- conversion ended -----------------
# of Home Services made:
11053
# converted...
111604
# unconverted..
45035
--------------- conversion ended -----------------
# of Health & Medical made:
8781
# converted...
120385
# unconverted..
36254
--------------- conversion ended -----------------
# of Nightlife made:
3800
# converted...
124185
# unconverted..
32454
--------------- conversion ended -----------------
# of Bars made:
0
# converted...
124185
# unconverted..
32454
--------------- conversion ended -----------------
# of Automotive made:
8038

* The regrouping demonstrated that only apprimately 2% of the data was unregrouped, which is not as significant so we ignored those. 

In [25]:
(len(businessDf)-2838)/len(businessDf)

0.9818819068048187

In [26]:
def convertToOther(df, columnName, dataType):
    notStringList =[]
    dataFrame = df[columnName]
    for observation in dataFrame:
        notString = False
        if(type(observation) != dataType):  
            notString = True
        notStringList.append(notString)
    
    notStringSeries = pd.Series(notStringList)
    return notStringSeries

In [27]:
businessDf['categories'].loc[convertToOther(businessDf, 'categories', str)]

19              [Financial Services, Banks & Credit Unions]
48              [Financial Services, Banks & Credit Unions]
66        [Departments of Motor Vehicles, Public Service...
94                          [Financial Services, Insurance]
119                                                      []
231             [Banks & Credit Unions, Financial Services]
248       [Financial Services, Insurance, Auto Insurance...
279             [Financial Services, Banks & Credit Unions]
282                               [Mass Media, Print Media]
299             [Banks & Credit Unions, Financial Services]
329                            [Radio Stations, Mass Media]
361                            [Radio Stations, Mass Media]
796             [Banks & Credit Unions, Financial Services]
812             [Banks & Credit Unions, Financial Services]
982                                                      []
1260                        [Insurance, Financial Services]
1332      [Landmarks & Historical Buildi

* We actually replaced anything money related with 'Financial Services' after some manual examination so that we only had ~1% of the data not regrouped in the 'categories' column

In [28]:
replaceCategory(businessDf, 'categories', 'Financial Services')
convertedCategories(businessDf, 'categories', str)

# of Financial Services made:
1251
# converted...
155052
# unconverted..
1587


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [29]:
(len(businessDf)-1587)/len(businessDf)

0.9898684235726735

We are only missing approximately 1% of the data. Thus, we will group the rest of the categories as 'Unknown' for the sake of convenience. 

In [30]:
businessDf['categories'].loc[convertToOther(businessDf, 'categories', str)] = 'Unknown'

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
convertedCategories(businessDf, 'categories', str) #we have completely converted all the categories

# converted...
156639
# unconverted..
0


In [32]:
businessDf['categories'].unique()

array(['Shopping', 'Restaurants', 'Food', 'Professional Services',
       'Health & Medical', 'Automotive', 'Active Life', 'Beauty & Spas',
       'Home Services', 'Financial Services', 'Local Services',
       'Nightlife', 'Arts & Entertainment', 'Unknown', 'Hotels & Travel',
       'Event Planning & Services', 'Education', 'Pets'], dtype=object)

## Part II: Unnesting JSON 
We decided to unnest the jsons of the attributes and hours info because those could be significant factors. While it does increase the number of factors significantly, this was the easiest way to parse the nested JSONs. In addition, our plan is to remove a lot of the columns because many of them have a lot of missing Data, so the dimensions won't be as big as the initial unnesting. 

In [33]:
def unnestJson(dataframe):
    data = list(dataframe)
    return json_normalize(data)

In [34]:
openHoursDf = unnestJson(businessDf['hours']) #The dataframe with unnested open hours info 
attributesDf = unnestJson(businessDf['attributes']) #The dataframe with unnested attributes info 

In [35]:
#merge two dataframes sidebyside by columns 
businessDf = pd.concat([businessDf, openHoursDf, attributesDf], axis=1) 

#len(businessDf)

In [36]:
#businessDf.to_csv("../dataset/unnested.csv")

The total number of dimensions currently is: 

In [37]:
len(businessDf.columns) 

103

# Dropping Repetitive, Too Detailed Columns 
* We do this because some of the columns' data is unnested and thus repetitive
* We also do this because we do not need as fine grain data (i.e. address) 

In [38]:
businessDf

address  \
0                                  691 Richmond Rd   
1                                   2824 Milton Rd   
2                              337 Danforth Avenue   
3              7702 E Doubletree Ranch Rd, Ste 300   
4                                   4719 N 20Th St   
5                              2017 E Camelback Rd   
6                          4425 N 24th St, Ste 125   
7                                   5770 Butler St   
8                               3220 Washington Rd   
9                        1425 S Higley Rd, Ste 103   
10                         11022 W Charleston Blvd   
11                         9393 N 90th St, Ste 112   
12                              9101 E Baseline Rd   
13                                 12417 W Pima St   
14                        9616 E Independence Blvd   
15                                 190 E Dallas Rd   
16                                        62 S Ctr   
17                        2128 Warrensville Ctr Rd   
18                                  3710 Payne Ave   
19                              2385 E Windmill Ln   
20                         10515 B Centrum Parkway   
21                                4759 Liberty Ave   
22                    7070 Saint Barbara Boulevard   
23                     1111 N Gilbert Rd, Ste 212A   
24                                                   
25                                 43 Leith Street   
26                             7 Hart House Circle   
27                                   1663 St-Denis   
28                                 6021 Centre Ave   
29                 15323 N. Scottsdale Rd, Ste 110   
...                                            ...   
156609                                 31 E 5th St   
156610                       1800 Las Vegas Blvd S   
156611                  4130 S Sandhill Rd, Ste A5   
156612                        401 E Bell Rd, Ste 5   
156613                          18 Tank House Lane   
156614                    4924 Sherbrooke Street W   
156615                             5680 S Pecos Rd   
156616                    5868 S Pecos Rd, Ste 200   
156617                   674-676 Sheppard Avenue W   
156618                 1371 Rue Sainte-Catherine E   
156619              12614 N Cave Creek Rd, Ste 108   
156620                      7101 W Craig Rd, Ste 1   
156621               15605 W Roosevelt St, Ste 106   
156622           2930 W Horizon Ridge Pky, Ste 100   
156623                  316 E Bridger Ave, Ste 101   
156624                             10659 Grand Ave   
156625                    9335 N Tryon St, Ste 102   
156626                               2401 Penn Ave   
156627                            16880 N 59th Ave   
156628        7700 W Arrowhead Towne Ctr, Ste 2049   
156629               6033 W Bell Rd, Bldg D, Ste F   
156630                 5293 Highway 7 E, Suite 204   
156631                         1925 St.Catherine W   
156632                           2419 Yonge Street   
156633                          9812 Rea Rd, Ste A   
156634                         1103 Queen Street E   
156635  Star Trek The Experience, 3000 Paradise Rd   
156636                               1833 E 3rd St   
156637                       The Croft Drumsmittal   
156638                                540 Marks St   

                                               attributes  \
0       {'RestaurantsPriceRange2': 2, 'BusinessParking...   
1       {'GoodForMeal': {'dessert': False, 'latenight'...   
2       {'BusinessParking': {'garage': False, 'street'...   
3                                                      {}   
4       {'RestaurantsTableService': False, 'GoodForMea...   
5       {'BusinessAcceptsCreditCards': True, 'Restaura...   
6       {'AcceptsInsurance': True, 'ByAppointmentOnly'...   
7                    {'BusinessAcceptsCreditCards': True}   
8       {'DogsAllowed': True, 'BusinessParking': {'gar...   
9       {'AcceptsInsurance': True, 'ByAppointmentOnly'...   
10      {'BusinessAccep

In [39]:
delCols1 = ['address', 'attributes', 'hours', 'latitude', 'longitude', 'name', 'postal_code']

In [40]:
businessDf = businessDf.drop( delCols1, axis=1) 

In [41]:
businessDf.head()

business_id             categories              city  is_open  \
0  YDf95gJZaq05wvo7hTQbbQ               Shopping  Richmond Heights        1   
1  mLwM-h2YhXl2NCgdS84_Bw            Restaurants         Charlotte        0   
2  v2WhjAB3PIBA8J8VxG3wEg                   Food           Toronto        0   
3  CVtCbSB1zUcUWg-9TNGTuQ  Professional Services        Scottsdale        1   
4  duHFBe87uNSXImQmvBh87Q            Restaurants           Phoenix        0   

  neighborhood  review_count  stars state       Friday       Monday  \
0                         17    2.0    OH  10:00-21:00  10:00-21:00   
1     Eastland             4    4.5    NC  10:00-22:00  10:00-22:00   
2    Riverdale             7    4.5    ON  10:00-19:00  10:00-19:00   
3                          3    3.0    AZ   9:00-17:00   9:00-17:00   
4                         10    4.5    AZ          NaN          NaN   

      Saturday       Sunday     Thursday      Tuesday    Wednesday  \
0  10:00-21:00  11:00-18:00  10:00-21:00  10:00-21:00  10:00-21:00   
1  10:00-22:00  10:00-22:00  10:00-22:00  10:00-22:00  10:00-22:00   
2  10:00-18:00  12:00-17:00  10:00-19:00  10:00-19:00  10:00-19:00   
3          NaN          NaN   9:00-17:00   9:00-17:00   9:00-17:00   
4          NaN          NaN          NaN          NaN          NaN   

  AcceptsInsurance AgesAllowed Alcohol Ambience.casual Ambience.classy  \
0              NaN         NaN     NaN             NaN             NaN   
1              NaN         NaN     NaN           False           False   
2              NaN         NaN     NaN             NaN             NaN   
3              NaN         NaN     NaN             NaN             NaN   
4              NaN         NaN    none           False           False   

  Ambience.divey Ambience.hipster Ambience.intimate Ambience.romantic  \
0            NaN              NaN               NaN               NaN   
1          False            False             False             False   
2            NaN              NaN               NaN               NaN   
3            NaN              NaN               NaN               NaN   
4          False            False             False             False   

  Ambience.touristy Ambience.trendy Ambience.upscale BYOB BYOBCorkage  \
0               NaN             NaN              NaN  NaN         NaN   
1             False           False            False  NaN         NaN   
2               NaN             NaN              NaN  NaN         NaN   
3               NaN             NaN              NaN  NaN         NaN   
4             False           False            False  NaN         NaN   

  BestNights.friday BestNights.monday BestNights.saturday BestNights.sunday  \
0               NaN               NaN                 NaN               NaN   
1               NaN               NaN                 NaN               NaN   
2               NaN               NaN                 NaN               NaN   
3               NaN               NaN                 NaN               NaN   
4               NaN               NaN                 NaN               NaN   

  BestNights.thursday BestNights.tuesday BestNights.wednesday BikeParking  \
0                 NaN                NaN                  NaN        True   
1                 NaN                NaN                  NaN         NaN   
2                 NaN                NaN                  NaN        True   
3                 NaN                NaN                  NaN         NaN   
4                 NaN                NaN                  NaN        True   

  BusinessAcceptsBitcoin BusinessAcceptsCreditCards BusinessParking.garage  \
0                    NaN                        NaN                  False   
1                    NaN                      False                    NaN   
2                    NaN                       True                  False   
3                    NaN                        NaN                    NaN   
4                    NaN                

# Looking At Missing Data Ratio
* We want to look at the number of missing data so we can get rid of those columns or dimensions that have too many missing values for the future
* However, before we delete columns, we decided to do some inspection on the restaurants/foods sectors or categories to see if the missing data ratio differed from the average missing data ratio of the entire dataset

In [42]:
 def missingDataSummary(df):
    missingDf = df.isnull().sum()/len(df)*100
    return pd.DataFrame(missingDf).T

In [43]:
missingData1 = missingDataSummary(businessDf)

In [44]:
missingData1

business_id  categories  city  is_open  neighborhood  review_count  stars  \
0          0.0         0.0   0.0      0.0           0.0           0.0    0.0   

   state     Friday   Monday   Saturday     Sunday   Thursday    Tuesday  \
0    0.0  27.629773  32.6879  35.424128  52.029827  27.356533  28.360753   

   Wednesday  AcceptsInsurance  AgesAllowed    Alcohol  Ambience.casual  \
0  27.648925         94.524991    99.747828  71.756714        72.612185   

   Ambience.classy  Ambience.divey  Ambience.hipster  Ambience.intimate  \
0        72.612185       82.649276         72.626868          72.612185   

   Ambience.romantic  Ambience.touristy  Ambience.trendy  Ambience.upscale  \
0          72.612185          72.612185        72.612185         72.612185   

        BYOB  BYOBCorkage  BestNights.friday  BestNights.monday  \
0  99.422238    99.106225          95.919918          95.919918   

   BestNights.saturday  BestNights.sunday  BestNights.thursday  \
0            95.919918          95.919918            95.919918   

   BestNights.tuesday  BestNights.wednesday  BikeParking  \
0           95.919918             95.919918     53.07682   

   BusinessAcceptsBitcoin  BusinessAcceptsCreditCards  BusinessParking.garage  \
0               94.333467                    22.91128               42.901193   

   BusinessParking.lot  BusinessParking.street  BusinessParking.valet  \
0            42.901193               42.901193              42.901193   

   BusinessParking.validated  ByAppointmentOnly     Caters  CoatCheck  \
0                  44.317188          77.794164  77.953766  94.908037   

     Corkage  DietaryRestrictions.dairy-free  DietaryRestrictions.gluten-free  \
0  99.583756                       99.841036                        99.841036   

   DietaryRestrictions.halal  DietaryRestrictions.kosher  \
0                  99.841036                   99.841036   

   DietaryRestrictions.soy-free  DietaryRestrictions.vegan  \
0                     99.841036                  99.841036   

   DietaryRestrictions.vegetarian  DogsAllowed  DriveThru  GoodForDancing  \
0                       99.841036    92.840863     96.144       94.552442   

   GoodForKids  GoodForMeal.breakfast  GoodForMeal.brunch  \
0     62.74108              72.578987           72.578987   

   GoodForMeal.dessert  GoodForMeal.dinner  GoodForMeal.latenight  \
0            72.578987           72.578987              72.578987   

   GoodForMeal.lunch  HairSpecializesIn.africanamerican  \
0          72.578987                          99.250506   

   HairSpecializesIn.asian  HairSpecializesIn.coloring  \
0                99.250506                   99.103033   

   HairSpecializesIn.curly  HairSpecializesIn.extensions  \
0                99.103033                     99.103033   

   HairSpecializesIn.kids  HairSpecializesIn.perms  \
0               99.103033                99.103033   

   HairSpecializesIn.straightperms  HappyHour      HasTV  \
0                        99.250506  94.627136  72.083581   

   Music.background_music   Music.dj  Music.jukebox  Music.karaoke  \
0               94.875478  94.875478      94.875478      94.875478   

   Music.live  Music.no_music  Music.video  NoiseLevel  Open24Hours  \
0   94.875478       94.875478    94.875478   74.046693    99.778472   

   OutdoorSeating  RestaurantsAttire  RestaurantsCounterService  \
0       67.881562          71.390905                  99.747189   

   RestaurantsDelivery  RestaurantsGoodForGroups  RestaurantsPriceRange2  \
0            70.120468                 68.316958               38.485945   

   RestaurantsReservations  RestaurantsTableService  RestaurantsTakeOut  \
0                70.299223                74.488473           65.112137   

     Smoking  WheelchairAccessible       WiFi  
0  95.101475              71.89525  72.274465

# Looking At Restaurants
* From the categories we made the assumption that the yelp dataset was most relevant regarding restaurant data
* In addition, we cared most about restaurants
* Thus, using 'categories' we will combine 'Restaurants' and 'Foods' and focus on that dataset 

In [45]:
businessDf['categories'].unique()

array(['Shopping', 'Restaurants', 'Food', 'Professional Services',
       'Health & Medical', 'Automotive', 'Active Life', 'Beauty & Spas',
       'Home Services', 'Financial Services', 'Local Services',
       'Nightlife', 'Arts & Entertainment', 'Unknown', 'Hotels & Travel',
       'Event Planning & Services', 'Education', 'Pets'], dtype=object)

In [46]:
restaurantsDf = businessDf.loc[(businessDf['categories'] == 'Restaurants')  ]

In [47]:
foodDf = businessDf.loc[(businessDf['categories'] == 'Food')  ]

In [48]:
missingDataRestaurants = missingDataSummary(restaurantsDf)

In [49]:
missingDataFood = missingDataSummary(foodDf)

In [50]:
missingDataRestaurants

business_id  categories  city  is_open  neighborhood  review_count  stars  \
0          0.0         0.0   0.0      0.0           0.0           0.0    0.0   

   state     Friday     Monday   Saturday     Sunday   Thursday    Tuesday  \
0    0.0  27.423324  34.179373  29.188383  37.988491  27.506636  28.957821   

   Wednesday  AcceptsInsurance  AgesAllowed    Alcohol  Ambience.casual  \
0   27.84376         99.980625       99.876  22.044834        22.771395   

   Ambience.classy  Ambience.divey  Ambience.hipster  Ambience.intimate  \
0        22.771395       51.136342          22.81402          22.771395   

   Ambience.romantic  Ambience.touristy  Ambience.trendy  Ambience.upscale  \
0          22.771395          22.771395        22.771395         22.771395   

        BYOB  BYOBCorkage  BestNights.friday  BestNights.monday  \
0  98.248503    97.287505          92.350764          92.350764   

   BestNights.saturday  BestNights.sunday  BestNights.thursday  \
0            92.350764          92.350764            92.350764   

   BestNights.tuesday  BestNights.wednesday  BikeParking  \
0           92.350764             92.350764    32.939376   

   BusinessAcceptsBitcoin  BusinessAcceptsCreditCards  BusinessParking.garage  \
0               95.257009                   11.681166               19.429214   

   BusinessParking.lot  BusinessParking.street  BusinessParking.valet  \
0            19.429214               19.429214              19.429214   

   BusinessParking.validated  ByAppointmentOnly     Caters  CoatCheck  \
0                  21.360897          99.083564  40.542111  90.297018   

     Corkage  DietaryRestrictions.dairy-free  DietaryRestrictions.gluten-free  \
0  98.736752                       99.527251                        99.527251   

   DietaryRestrictions.halal  DietaryRestrictions.kosher  \
0                  99.527251                   99.527251   

   DietaryRestrictions.soy-free  DietaryRestrictions.vegan  \
0                     99.527251                  99.527251   

   DietaryRestrictions.vegetarian  DogsAllowed  DriveThru  GoodForDancing  \
0                       99.527251      86.6119  88.301397       89.430957   

   GoodForKids  GoodForMeal.breakfast  GoodForMeal.brunch  \
0    11.967915              17.809467           17.809467   

   GoodForMeal.dessert  GoodForMeal.dinner  GoodForMeal.latenight  \
0            17.809467           17.809467              17.809467   

   GoodForMeal.lunch  HairSpecializesIn.africanamerican  \
0          17.809467                          99.994188   

   HairSpecializesIn.asian  HairSpecializesIn.coloring  \
0                99.994188                   99.990313   

   HairSpecializesIn.curly  HairSpecializesIn.extensions  \
0                99.990313                     99.990313   

   HairSpecializesIn.kids  HairSpecializesIn.perms  \
0               99.990313                99.990313   

   HairSpecializesIn.straightperms  HappyHour     HasTV  \
0                        99.994188  89.682832  21.40546   

   Music.background_music   Music.dj  Music.jukebox  Music.karaoke  \
0               90.244706  90.244706      90.244706      90.244706   

   Music.live  Music.no_music  Music.video  NoiseLevel  Open24Hours  \
0   90.244706       90.244706    90.244706   26.694825    99.471063   

   OutdoorSeating  RestaurantsAttire  RestaurantsCounterService  \
0       14.682347          13.204038                  99.232751   

   RestaurantsDelivery  RestaurantsGoodForGroups  RestaurantsPriceRange2  \
0            13.692287                 10.836417                8.329297   

   RestaurantsReservations  RestaurantsTableService  RestaurantsTakeOut  \
0                13.089725                 22.58927            9.846357   

     Smoking  WheelchairAccessible       WiFi  
0  90.707767              58.65189  31.205316

In [51]:
missingDataFood

business_id  categories  city  is_open  neighborhood  review_count  stars  \
0          0.0         0.0   0.0      0.0           0.0           0.0    0.0   

   state     Friday     Monday   Saturday     Sunday   Thursday   Tuesday  \
0    0.0  35.430751  41.697001  36.860808  46.455192  35.794765  37.17282   

   Wednesday  AcceptsInsurance  AgesAllowed    Alcohol  Ambience.casual  \
0  35.950771         99.774658    99.973999  92.927717        97.876582   

   Ambience.classy  Ambience.divey  Ambience.hipster  Ambience.intimate  \
0        97.876582       98.483273         97.876582          97.876582   

   Ambience.romantic  Ambience.touristy  Ambience.trendy  Ambience.upscale  \
0          97.876582          97.876582        97.876582         97.876582   

    BYOB  BYOBCorkage  BestNights.friday  BestNights.monday  \
0  100.0        100.0          98.552609          98.552609   

   BestNights.saturday  BestNights.sunday  BestNights.thursday  \
0            98.552609          98.552609            98.552609   

   BestNights.tuesday  BestNights.wednesday  BikeParking  \
0           98.552609             98.552609     27.82978   

   BusinessAcceptsBitcoin  BusinessAcceptsCreditCards  BusinessParking.garage  \
0               96.593864                   10.721096               12.870515   

   BusinessParking.lot  BusinessParking.street  BusinessParking.valet  \
0            12.870515               12.870515              12.870515   

   BusinessParking.validated  ByAppointmentOnly     Caters  CoatCheck  \
0                   15.49662           96.25585  71.858208  98.101924   

   Corkage  DietaryRestrictions.dairy-free  DietaryRestrictions.gluten-free  \
0    100.0                       99.956665                        99.956665   

   DietaryRestrictions.halal  DietaryRestrictions.kosher  \
0                  99.956665                   99.956665   

   DietaryRestrictions.soy-free  DietaryRestrictions.vegan  \
0                     99.956665                  99.956665   

   DietaryRestrictions.vegetarian  DogsAllowed  DriveThru  GoodForDancing  \
0                       99.956665    97.105218      100.0       97.893916   

   GoodForKids  GoodForMeal.breakfast  GoodForMeal.brunch  \
0    98.162593              96.065176           96.065176   

   GoodForMeal.dessert  GoodForMeal.dinner  GoodForMeal.latenight  \
0            96.065176           96.065176              96.065176   

   GoodForMeal.lunch  HairSpecializesIn.africanamerican  \
0          96.065176                          99.991333   

   HairSpecializesIn.asian  HairSpecializesIn.coloring  \
0                99.991333                   99.991333   

   HairSpecializesIn.curly  HairSpecializesIn.extensions  \
0                99.991333                     99.991333   

   HairSpecializesIn.kids  HairSpecializesIn.perms  \
0               99.991333                99.991333   

   HairSpecializesIn.straightperms  HappyHour      HasTV  \
0                        99.991333  97.937251  97.807246   

   Music.background_music   Music.dj  Music.jukebox  Music.karaoke  \
0               98.145259  98.145259      98.145259      98.145259   

   Music.live  Music.no_music  Music.video  NoiseLevel  Open24Hours  \
0   98.145259       98.145259    98.145259   98.127925        100.0   

   OutdoorSeating  RestaurantsAttire  RestaurantsCounterService  \
0       74.059629          99.982666                      100.0   

   RestaurantsDelivery  RestaurantsGoodForGroups  RestaurantsPriceRange2  \
0            86.054776                 97.607904                6.179581   

   RestaurantsReservations  RestaurantsTableService  RestaurantsTakeOut  \
0                98.457272                99.991333           39.521581   

     Smoking  WheelchairAccessible       WiFi  
0  98.275264             65.453285  64.517247

In [52]:
restaurantDf = pd.concat([restaurantsDf, foodDf], axis = 0).reset_index()

In [53]:
restaurantDf = restaurantDf.drop(['categories'], axis = 1) #drop it because we now know that this dataset only includes restaurant information

In [54]:
restaurantDf

index             business_id           city  is_open  \
0           1  mLwM-h2YhXl2NCgdS84_Bw      Charlotte        0   
1           4  duHFBe87uNSXImQmvBh87Q        Phoenix        0   
2          14  SDMRxmcKPNt1AHPBKqO64Q       Matthews        1   
3          15  iFEiMJoEqyB9O8OUNSdLzA        Stanley        1   
4          21  HmI9nhgOkrXlUr6KZGZZew     Pittsburgh        1   
5          22  qnpvw-uQyRn9nlClWFK9aA    Mississauga        1   
6          31  TXiEgINSZ75d3EtvLvkc4Q        Madison        1   
7          32  KW4y7uDGjVfU3ClkEjIGhg      Pickering        1   
8          34  reWc1g65PNZnKz_Ub9QKOQ  Richmond Hill        1   
9          35  L1XHTn7S-6har9UGAPjcWQ  Richmond Hill        1   
10         36  PV9CdNFDOX4_zWm3Sy3W8g      Middleton        1   
11         38  qim0lD112TkDhm8ZyQlRnA        Toronto        0   
12         42  HRFJlSAP_EBU_MpPPmpUDQ       Chandler        1   
13         50  FLTbKRmBytP1AbjWSPEeyw       Chandler        0   
14         51  1AxEmgv8Dsr3iU9Aa40jPw       Chandler        0   
15         53  Wf5C8Amv_SlhoYE3_W66WQ        Toronto        0   
16         54  58APdML-PG_OD4El2ePTvw       Montréal        1   
17         55  Z1r6b30Tg0n0ME4-Zj2wQQ        Toronto        1   
18         56  -2q4dnUw0gGJniGW2aPamQ      Champaign        0   
19         58  LDMCrFlGIFUN6L-FEFgzWg      Las Vegas        1   
20         59  cBHMUESPj4SNs65Xv6xWRA      Edinburgh        1   
21         64  QTH_XGh4rWYdd0fTW-tUDw      Las Vegas        0   
22         70  01xXe2m_z048W5gcBFpoJA        Phoenix        1   
23         71  dEmNOTm8Rmm9JYZdGX_Lhw       Montreal        1   
24         79  Bl7Y-ATTzXytQnCceg5k6w    Painesville        1   
25         80  BvCHyg0GtxA6XKmRC0cQsg    Mississauga        1   
26         82  fl2TPNWrchkCbNEg0utjvw         Urbana        1   
27         86  b5VIe-VnkOOwiwkjGw071A        Glasgow        1   
28         90  DiA78qPtp6rfRNdomzjBbw      Las Vegas        1   
29         98  d2fkRF67jiASrXxHfbmJuA       Chandler        1   
...       ...                     ...            ...      ...   
63121  156371  pzcqjJ3iBfjQyt8PzgHsYQ      Las Vegas        1   
63122  156372  4OKs1apDDZ2qvgdz5AXDhw      Las Vegas        1   
63123  156396  5J3b7j3Fzo9ISjChmoUoUA        Toronto        1   
63124  156397  qNfZ-qKq92kPrDYNcexZ9Q       Montréal        1   
63125  156418  4qtumuLnILMkhWI9aIXJwQ      Las Vegas        0   
63126  156420  iNWx7Wt5w0ySf8BrLlolIg        Toronto        1   
63127  156423  vDZlLRiqIjcMe_h3COKHlg      Charlotte        1   
63128  156432  mA9q_LYOicEffXDIqSjySw        Markham        1   
63129  156434  kBt12nJhnruF9NQ36vRSZA        Toronto        1   
63130  156436  HzhPJ6QJsWX-lkfj8y9s9Q        Toronto        0   
63131  156450  rMVcz8N6y_da9VbBgVw3XA        Toronto        1   
63132  156461  F6gzqYbLjow9ZgHW3ROxcA        Madison        1   
63133  156471  MUS89o9JnvfqsDhFcW1Cfg      Sewickley        1   
63134  156473  -G9DFmuokdYk_H35ezRkAw      Las Vegas        1   
63135  156496  8gPvDLHY6D7NjJxpikUc6Q        Toronto        0   
63136  156499  zYXG5RsbxiOkZeJ7yAEj3Q        Toronto        1   
63137  156500  4kkAIeRcRKpMN32jVksLEw         Lowell        0   
63138  156505  8UzfIr1PtgxadKJ53dfphA    Filderstadt        1   
63139  156510  7luN-NaZE659McLI_Apcyw    Mississauga        1   
63140  156532  UFa4Vz3i2pvc6viF_W6lPQ       Montréal        1   
63141  156539  u4jR8L7hMFrhxrZq2NNTmg     Pittsburgh        1   
63142  156564  o3eYn24dSFeu2tPD9D4eMQ        Toronto        1   
63143  156565  dQAcT0i8dqlzK5YMa1bWDQ       Montréal        1   
63144  156569  UkM9xM8SkHzMe_x5zl0MhQ     Pittsburgh        1   
63145  156575  6Du78g14y2MyDSEWxcwOmw        Belmont        1   
63146  156578  BAtK_WOxicYU3w42wPJYXw           Mesa        1   
63147  156583  4iSVxQKK1G3V-LBffQxaKg     Pittsburgh        1   
63148  156606  nQk2Xy_HlAmZCyYvgLpuZw      Henderson        1   
63149  156608  3nooaG-Tik4WzPxU43Bm9A      Las Vegas        0   
63150  156620  oVji4RtNNaGJU119CaCU1w      

In [55]:
missingDataRestaurant = missingDataSummary(restaurantDf)
missingDataRestaurant

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state    Friday     Monday   Saturday     Sunday   Thursday    Tuesday  \
0    0.0  28.88632  35.552881  30.590173  39.535399  29.020918  30.458742   

   Wednesday  AcceptsInsurance  AgesAllowed    Alcohol  Ambience.casual  \
0  29.324951         99.942994    99.893905  34.995487        36.493484   

   Ambience.classy  Ambience.divey  Ambience.hipster  Ambience.intimate  \
0        36.493484        59.78686         36.528321          36.493484   

   Ambience.romantic  Ambience.touristy  Ambience.trendy  Ambience.upscale  \
0          36.493484          36.493484        36.493484         36.493484   

       BYOB  BYOBCorkage  BestNights.friday  BestNights.monday  \
0  98.56851    97.783091          93.483872          93.483872   

   BestNights.saturday  BestNights.sunday  BestNights.thursday  \
0            93.483872          93.483872            93.483872   

   BestNights.tuesday  BestNights.wednesday  BikeParking  \
0           93.483872             93.483872    32.005827   

   BusinessAcceptsBitcoin  BusinessAcceptsCreditCards  BusinessParking.garage  \
0               95.501259                   11.505756               18.230907   

   BusinessParking.lot  BusinessParking.street  BusinessParking.valet  \
0            18.230907               18.230907              18.230907   

   BusinessParking.validated  ByAppointmentOnly     Caters  CoatCheck  \
0                  20.289465          98.566927  46.263717  91.723013   

     Corkage  DietaryRestrictions.dairy-free  DietaryRestrictions.gluten-free  \
0  98.967554                       99.605707                        99.605707   

   DietaryRestrictions.halal  DietaryRestrictions.kosher  \
0                  99.605707                   99.605707   

   DietaryRestrictions.soy-free  DietaryRestrictions.vegan  \
0                     99.605707                  99.605707   

   DietaryRestrictions.vegetarian  DogsAllowed  DriveThru  GoodForDancing  \
0                       99.605707    88.529081   90.43879       90.977182   

   GoodForKids  GoodForMeal.breakfast  GoodForMeal.brunch  \
0    27.716109              32.107172           32.107172   

   GoodForMeal.dessert  GoodForMeal.dinner  GoodForMeal.latenight  \
0            32.107172           32.107172              32.107172   

   GoodForMeal.lunch  HairSpecializesIn.africanamerican  \
0          32.107172                          99.993666   

   HairSpecializesIn.asian  HairSpecializesIn.coloring  \
0                99.993666                   99.990499   

   HairSpecializesIn.curly  HairSpecializesIn.extensions  \
0                99.990499                     99.990499   

   HairSpecializesIn.kids  HairSpecializesIn.perms  \
0               99.990499                99.990499   

   HairSpecializesIn.straightperms  HappyHour      HasTV  \
0                        99.993666  91.190955  35.364444   

   Music.background_music   Music.dj  Music.jukebox  Music.karaoke  \
0               91.688176  91.688176      91.688176      91.688176   

   Music.live  Music.no_music  Music.video  NoiseLevel  Open24Hours  \
0   91.688176       91.688176    91.688176   39.746006    99.567703   

   OutdoorSeating  RestaurantsAttire  RestaurantsCounterService  \
0        25.53087          29.058922                  99.372932   

   RestaurantsDelivery  RestaurantsGoodForGroups  RestaurantsPriceRange2  \
0            26.913271                 26.689997                7.936533   

   RestaurantsReservations  RestaurantsTableService  RestaurantsTakeOut  \
0                28.686798                 36.73101           15.268167   

     Smoking  WheelchairAccessible       WiFi  
0  92.090387             59.894538  37.291571

# Factors Completely not important for Restaurants 
* We are looking into the columns 'not important' or the ones with the most missing data 
* Determining which columns to delete was actually an iterative process. We continuously modeled and saw if we had enough data. From practice, it seemed that if there were more than 20% of the data missing, it was hard to create a comprehensible decision tree because it would split into Unknown too frequently to provide much meaningful data. Additional complications seemed to arise because we had too few observations such that having a 1/3 of its data as Unknown was hard to provide analysis on. 
* If time provides we might change the categorizaiton of our data and maybe "improve" our model by including data points that have more than 30% of the data missing. 

In [56]:
def colsMissing(df, percentage):
    missingCols = []
    for column in df.columns:
        if(df[column][0] >percentage):
            missingCols.append(column)
    
    return missingCols

In [57]:
missing90DataCols = colsMissing(missingDataRestaurant, 90)

These are the columns that really don't matter in restaurants:

In [58]:
missing90DataCols

['AcceptsInsurance',
 'AgesAllowed',
 'BYOB',
 'BYOBCorkage',
 'BestNights.friday',
 'BestNights.monday',
 'BestNights.saturday',
 'BestNights.sunday',
 'BestNights.thursday',
 'BestNights.tuesday',
 'BestNights.wednesday',
 'BusinessAcceptsBitcoin',
 'ByAppointmentOnly',
 'CoatCheck',
 'Corkage',
 'DietaryRestrictions.dairy-free',
 'DietaryRestrictions.gluten-free',
 'DietaryRestrictions.halal',
 'DietaryRestrictions.kosher',
 'DietaryRestrictions.soy-free',
 'DietaryRestrictions.vegan',
 'DietaryRestrictions.vegetarian',
 'DriveThru',
 'GoodForDancing',
 'HairSpecializesIn.africanamerican',
 'HairSpecializesIn.asian',
 'HairSpecializesIn.coloring',
 'HairSpecializesIn.curly',
 'HairSpecializesIn.extensions',
 'HairSpecializesIn.kids',
 'HairSpecializesIn.perms',
 'HairSpecializesIn.straightperms',
 'HappyHour',
 'Music.background_music',
 'Music.dj',
 'Music.jukebox',
 'Music.karaoke',
 'Music.live',
 'Music.no_music',
 'Music.video',
 'Open24Hours',
 'RestaurantsCounterService',
 'S

We deleted the following number of columns:

In [59]:
len(missing90DataCols)

43

Something surprising is that 'Open24Hours',  'HappyHour', 'Music', and  'DriveThru' does not seem to 'matter' for restaurants (in the sense that their is not information on those features). This is interesting, because these are generally restaurants' selling points or marketing tactics.

In [60]:
restaurantDf = restaurantDf.drop( missing90DataCols, axis=1) 

The dimensions of the restaurant dataset is now: 

In [61]:
len(restaurantDf.columns) 

53

In [62]:
missingDataRestaurant = missingDataSummary(restaurantDf)
missingDataRestaurant

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state    Friday     Monday   Saturday     Sunday   Thursday    Tuesday  \
0    0.0  28.88632  35.552881  30.590173  39.535399  29.020918  30.458742   

   Wednesday    Alcohol  Ambience.casual  Ambience.classy  Ambience.divey  \
0  29.324951  34.995487        36.493484        36.493484        59.78686   

   Ambience.hipster  Ambience.intimate  Ambience.romantic  Ambience.touristy  \
0         36.528321          36.493484          36.493484          36.493484   

   Ambience.trendy  Ambience.upscale  BikeParking  BusinessAcceptsCreditCards  \
0        36.493484         36.493484    32.005827                   11.505756   

   BusinessParking.garage  BusinessParking.lot  BusinessParking.street  \
0               18.230907            18.230907               18.230907   

   BusinessParking.valet  BusinessParking.validated     Caters  DogsAllowed  \
0              18.230907                  20.289465  46.263717    88.529081   

   GoodForKids  GoodForMeal.breakfast  GoodForMeal.brunch  \
0    27.716109              32.107172           32.107172   

   GoodForMeal.dessert  GoodForMeal.dinner  GoodForMeal.latenight  \
0            32.107172           32.107172              32.107172   

   GoodForMeal.lunch      HasTV  NoiseLevel  OutdoorSeating  \
0          32.107172  35.364444   39.746006        25.53087   

   RestaurantsAttire  RestaurantsDelivery  RestaurantsGoodForGroups  \
0          29.058922            26.913271                 26.689997   

   RestaurantsPriceRange2  RestaurantsReservations  RestaurantsTableService  \
0                7.936533                28.686798                 36.73101   

   RestaurantsTakeOut  WheelchairAccessible       WiFi  
0           15.268167             59.894538  37.291571

In [63]:
missing80DataCols = colsMissing(missingDataRestaurant, 80)
missing80DataCols

['DogsAllowed']

We deleted the following number of columns:

In [64]:
len(missing80DataCols)

1

In [65]:
restaurantDf = restaurantDf.drop( missing80DataCols, axis=1) 

In [66]:
missingDataRestaurant = missingDataSummary(restaurantDf)
missingDataRestaurant

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state    Friday     Monday   Saturday     Sunday   Thursday    Tuesday  \
0    0.0  28.88632  35.552881  30.590173  39.535399  29.020918  30.458742   

   Wednesday    Alcohol  Ambience.casual  Ambience.classy  Ambience.divey  \
0  29.324951  34.995487        36.493484        36.493484        59.78686   

   Ambience.hipster  Ambience.intimate  Ambience.romantic  Ambience.touristy  \
0         36.528321          36.493484          36.493484          36.493484   

   Ambience.trendy  Ambience.upscale  BikeParking  BusinessAcceptsCreditCards  \
0        36.493484         36.493484    32.005827                   11.505756   

   BusinessParking.garage  BusinessParking.lot  BusinessParking.street  \
0               18.230907            18.230907               18.230907   

   BusinessParking.valet  BusinessParking.validated     Caters  GoodForKids  \
0              18.230907                  20.289465  46.263717    27.716109   

   GoodForMeal.breakfast  GoodForMeal.brunch  GoodForMeal.dessert  \
0              32.107172           32.107172            32.107172   

   GoodForMeal.dinner  GoodForMeal.latenight  GoodForMeal.lunch      HasTV  \
0           32.107172              32.107172          32.107172  35.364444   

   NoiseLevel  OutdoorSeating  RestaurantsAttire  RestaurantsDelivery  \
0   39.746006        25.53087          29.058922            26.913271   

   RestaurantsGoodForGroups  RestaurantsPriceRange2  RestaurantsReservations  \
0                 26.689997                7.936533                28.686798   

   RestaurantsTableService  RestaurantsTakeOut  WheelchairAccessible  \
0                 36.73101           15.268167             59.894538   

        WiFi  
0  37.291571

In [67]:
missing40DataCols = colsMissing(missingDataRestaurant, 40)
missing40DataCols

['Ambience.divey', 'Caters', 'WheelchairAccessible']

We deleted the following number of columns: 

In [68]:
len(missing40DataCols)

3

While it is reasonable that these are important for choosing a restaurant, we already have enough dimensions so we decided to drop them. This is sort of unfortunate because this may sort of show that we are deleting info important for a subset group of people, those who need wheel chair, not because we are intentionally trying to bias our results or data not in favor of those people but probably because the way that the data was collected. I mean this in the sens that not a lot of data was collected regarding wheel chair support information. Thus, it cannot be within our data... ? 

In [69]:
restaurantDf = restaurantDf.drop( missing40DataCols, axis=1) 

These are the new # dimensions for the restaurant dataset 

In [70]:
len(restaurantDf.columns)

49

In [71]:
missingDataRestaurant = missingDataSummary(restaurantDf)
missingDataRestaurant

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state    Friday     Monday   Saturday     Sunday   Thursday    Tuesday  \
0    0.0  28.88632  35.552881  30.590173  39.535399  29.020918  30.458742   

   Wednesday    Alcohol  Ambience.casual  Ambience.classy  Ambience.hipster  \
0  29.324951  34.995487        36.493484        36.493484         36.528321   

   Ambience.intimate  Ambience.romantic  Ambience.touristy  Ambience.trendy  \
0          36.493484          36.493484          36.493484        36.493484   

   Ambience.upscale  BikeParking  BusinessAcceptsCreditCards  \
0         36.493484    32.005827                   11.505756   

   BusinessParking.garage  BusinessParking.lot  BusinessParking.street  \
0               18.230907            18.230907               18.230907   

   BusinessParking.valet  BusinessParking.validated  GoodForKids  \
0              18.230907                  20.289465    27.716109   

   GoodForMeal.breakfast  GoodForMeal.brunch  GoodForMeal.dessert  \
0              32.107172           32.107172            32.107172   

   GoodForMeal.dinner  GoodForMeal.latenight  GoodForMeal.lunch      HasTV  \
0           32.107172              32.107172          32.107172  35.364444   

   NoiseLevel  OutdoorSeating  RestaurantsAttire  RestaurantsDelivery  \
0   39.746006        25.53087          29.058922            26.913271   

   RestaurantsGoodForGroups  RestaurantsPriceRange2  RestaurantsReservations  \
0                 26.689997                7.936533                28.686798   

   RestaurantsTableService  RestaurantsTakeOut       WiFi  
0                 36.73101           15.268167  37.291571

In [72]:
missing30DataCols = colsMissing(missingDataRestaurant, 30)
missing30DataCols

['Monday',
 'Saturday',
 'Sunday',
 'Tuesday',
 'Alcohol',
 'Ambience.casual',
 'Ambience.classy',
 'Ambience.hipster',
 'Ambience.intimate',
 'Ambience.romantic',
 'Ambience.touristy',
 'Ambience.trendy',
 'Ambience.upscale',
 'BikeParking',
 'GoodForMeal.breakfast',
 'GoodForMeal.brunch',
 'GoodForMeal.dessert',
 'GoodForMeal.dinner',
 'GoodForMeal.latenight',
 'GoodForMeal.lunch',
 'HasTV',
 'NoiseLevel',
 'RestaurantsTableService',
 'WiFi']

We deleted the following number of columns:

In [73]:
len(missing30DataCols)

24

In [74]:
restaurantDf = restaurantDf.drop( missing30DataCols, axis=1) 

In [75]:
missingData_30percent = missingDataSummary(restaurantDf)

In [76]:
missingData_30percent

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state    Friday   Thursday  Wednesday  BusinessAcceptsCreditCards  \
0    0.0  28.88632  29.020918  29.324951                   11.505756   

   BusinessParking.garage  BusinessParking.lot  BusinessParking.street  \
0               18.230907            18.230907               18.230907   

   BusinessParking.valet  BusinessParking.validated  GoodForKids  \
0              18.230907                  20.289465    27.716109   

   OutdoorSeating  RestaurantsAttire  RestaurantsDelivery  \
0        25.53087          29.058922            26.913271   

   RestaurantsGoodForGroups  RestaurantsPriceRange2  RestaurantsReservations  \
0                 26.689997                7.936533                28.686798   

   RestaurantsTakeOut  
0           15.268167

In [77]:
missing20DataCols = colsMissing(missingData_30percent, 20)
missing20DataCols

['Friday',
 'Thursday',
 'Wednesday',
 'BusinessParking.validated',
 'GoodForKids',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsReservations']

We deleted the following number of columns:

In [78]:
len(missing20DataCols)

10

In [79]:
restaurantDf = restaurantDf.drop( missing20DataCols, axis=1) 

In [80]:
missingData_20percent = missingDataSummary(restaurantDf)

In [81]:
missingData_20percent

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state  BusinessAcceptsCreditCards  BusinessParking.garage  \
0    0.0                   11.505756               18.230907   

   BusinessParking.lot  BusinessParking.street  BusinessParking.valet  \
0            18.230907               18.230907              18.230907   

   RestaurantsPriceRange2  RestaurantsTakeOut  
0                7.936533           15.268167

Now we have 14 columns in our dataset that would define properties of a restaurant:

In [84]:
restaurantDf.columns

Index(['index', 'business_id', 'city', 'is_open', 'neighborhood',
       'review_count', 'stars', 'state', 'BusinessAcceptsCreditCards',
       'BusinessParking.garage', 'BusinessParking.lot',
       'BusinessParking.street', 'BusinessParking.valet',
       'RestaurantsPriceRange2', 'RestaurantsTakeOut'],
      dtype='object')

# Merging/Grouping Parking Data 
* If we know that there is parking in the garge, lot, street, or valet, we indicate that there is parking. Else, we say false. 
* This process eventually allowed us to merge four columns: 'BusinessParking.garage', 'BusinessParking.lot', 'BusinessParking.street', 'BusinessParking.valet' into one.
* Even if it is unknown whether there exists parking we say false because we believed that it was better to assume there was no parking. We decided to do this because after creating some decision tree models with "Unknown" labels, we concluded that 20% of them being "Unknowns" was too many and made the model incomprehnsible. Also, this had around 20% of the data missing, so filling missing data with "Unknown"s we believed would complicate our model too much.
* In addition, this was just one method to deal with missing data. We also believed that it was a fair assumption, based on our experiences, that people assumed there was no parking if the Yelp business info had no information on Parking, particularly because 'Parking' is a selling point for restaurants. So, if the restaurant had parking, it would probably want to advertise that and list that in their information. Thus, the fact that the restaurant had missing parking data seemed to indicate to us that the restaurant either did not care about its parking info, was unaware of the category, or did not want to list information on it because it had no parking. 
*  We also believed that this regrouping of missing data to 'false' was justified because it's better for a user to assume that there is no parking if there is no information on it than to assume that a restaurant does. 

In [79]:
parkingCols = []
for column in restaurantDf.columns:
    if "BusinessParking" in column:
        parkingCols.append(column)

In [80]:
parkingCols

['BusinessParking.garage',
 'BusinessParking.lot',
 'BusinessParking.street',
 'BusinessParking.valet']

In [81]:
parkingDf = restaurantDf[parkingCols]

In [82]:
parkingList = []
for i in range(len(parkingDf)):
    if((parkingDf.iloc[i] == True).any()):
        parkingList.append(True)
    else:
        parkingList.append(False) 

In [83]:
newParkingDf = pd.DataFrame(parkingList)
newParkingDf.columns = ['Parking']

In [84]:
restaurantDf = pd.concat([restaurantDf, newParkingDf], axis = 1)

In [85]:
restaurantDf.head()

index             business_id        city  is_open neighborhood  \
0      1  mLwM-h2YhXl2NCgdS84_Bw   Charlotte        0     Eastland   
1      4  duHFBe87uNSXImQmvBh87Q     Phoenix        0                
2     14  SDMRxmcKPNt1AHPBKqO64Q    Matthews        1                
3     15  iFEiMJoEqyB9O8OUNSdLzA     Stanley        1                
4     21  HmI9nhgOkrXlUr6KZGZZew  Pittsburgh        1   Bloomfield   

   review_count  stars state BusinessAcceptsCreditCards  \
0             4    4.5    NC                      False   
1            10    4.5    AZ                       True   
2            21    2.0    NC                       True   
3             3    3.0    NC                        NaN   
4            15    3.0    PA                       True   

  BusinessParking.garage BusinessParking.lot BusinessParking.street  \
0                    NaN                 NaN                    NaN   
1                  False               False                  False   
2                  False               False                  False   
3                    NaN                 NaN                    NaN   
4                  False               False                   True   

  BusinessParking.valet  RestaurantsPriceRange2 RestaurantsTakeOut  Parking  
0                   NaN                     2.0               True    False  
1                 False                     1.0               True    False  
2                 False                     2.0               True    False  
3                   NaN                     NaN                NaN    False  
4                 False                     1.0               True     True

In [86]:
restaurantDf = restaurantDf.drop(parkingCols, axis = 1)

In [87]:
restaurantDf.head()

index             business_id        city  is_open neighborhood  \
0      1  mLwM-h2YhXl2NCgdS84_Bw   Charlotte        0     Eastland   
1      4  duHFBe87uNSXImQmvBh87Q     Phoenix        0                
2     14  SDMRxmcKPNt1AHPBKqO64Q    Matthews        1                
3     15  iFEiMJoEqyB9O8OUNSdLzA     Stanley        1                
4     21  HmI9nhgOkrXlUr6KZGZZew  Pittsburgh        1   Bloomfield   

   review_count  stars state BusinessAcceptsCreditCards  \
0             4    4.5    NC                      False   
1            10    4.5    AZ                       True   
2            21    2.0    NC                       True   
3             3    3.0    NC                        NaN   
4            15    3.0    PA                       True   

   RestaurantsPriceRange2 RestaurantsTakeOut  Parking  
0                     2.0               True    False  
1                     1.0               True    False  
2                     2.0               True    False  
3                     NaN                NaN    False  
4                     1.0               True     True

In [88]:
missingDataInfo = missingDataSummary(restaurantDf)
missingDataInfo

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state  BusinessAcceptsCreditCards  RestaurantsPriceRange2  \
0    0.0                   11.505756                7.936533   

   RestaurantsTakeOut  Parking  
0           15.268167      0.0

# Filling Missing Data 
* The rest of the dataset will be filled with 'Unknown' if the data is missing 
* This will reduce the bias of setting the missing data values as some arbitrary value 
* Conclusively, we decided that 15% missing data is acceptable to have the three labels: "True", "False", and "Unknown"

In [89]:
missing0DataCols = colsMissing(missingDataInfo, 0)
missing0DataCols

['BusinessAcceptsCreditCards', 'RestaurantsPriceRange2', 'RestaurantsTakeOut']

In [90]:
for column in missing0DataCols: 
    restaurantDf[column].fillna('Unknown', inplace = True)

In [91]:
restaurantDf.head()

index             business_id        city  is_open neighborhood  \
0      1  mLwM-h2YhXl2NCgdS84_Bw   Charlotte        0     Eastland   
1      4  duHFBe87uNSXImQmvBh87Q     Phoenix        0                
2     14  SDMRxmcKPNt1AHPBKqO64Q    Matthews        1                
3     15  iFEiMJoEqyB9O8OUNSdLzA     Stanley        1                
4     21  HmI9nhgOkrXlUr6KZGZZew  Pittsburgh        1   Bloomfield   

   review_count  stars state BusinessAcceptsCreditCards  \
0             4    4.5    NC                      False   
1            10    4.5    AZ                       True   
2            21    2.0    NC                       True   
3             3    3.0    NC                    Unknown   
4            15    3.0    PA                       True   

  RestaurantsPriceRange2 RestaurantsTakeOut  Parking  
0                      2               True    False  
1                      1               True    False  
2                      2               True    False  
3                Unknown            Unknown    False  
4                      1               True     True

In [92]:
missingDataRestaurant = missingDataSummary(restaurantDf)
missingDataRestaurant

index  business_id  city  is_open  neighborhood  review_count  stars  \
0    0.0          0.0   0.0      0.0           0.0           0.0    0.0   

   state  BusinessAcceptsCreditCards  RestaurantsPriceRange2  \
0    0.0                         0.0                     0.0   

   RestaurantsTakeOut  Parking  
0                 0.0      0.0

In [93]:
restaurantDf['city'] = restaurantDf['city'].replace([''], ['Unknown'])
restaurantDf['neighborhood'] = restaurantDf['neighborhood'].replace([''], ['Unknown'])

Conclusively we have 12- 1 = 11 columns/properties that describe our restaurant data set. 

In [94]:
len(restaurantDf.columns)

12

# Converting Data Type 
We converted the binary 1/0 data type to "Open" and "Closed" for the is_open column. This was just for convenience when we export it to R so that R understands the 'is_open' column as one with categorical data types. 

In [95]:
restaurantDf['is_open'] = restaurantDf['is_open'].map({0 : "Closed", 1: "Open"})

In [96]:
restaurantDf.head()

index             business_id        city is_open neighborhood  \
0      1  mLwM-h2YhXl2NCgdS84_Bw   Charlotte  Closed     Eastland   
1      4  duHFBe87uNSXImQmvBh87Q     Phoenix  Closed      Unknown   
2     14  SDMRxmcKPNt1AHPBKqO64Q    Matthews    Open      Unknown   
3     15  iFEiMJoEqyB9O8OUNSdLzA     Stanley    Open      Unknown   
4     21  HmI9nhgOkrXlUr6KZGZZew  Pittsburgh    Open   Bloomfield   

   review_count  stars state BusinessAcceptsCreditCards  \
0             4    4.5    NC                      False   
1            10    4.5    AZ                       True   
2            21    2.0    NC                       True   
3             3    3.0    NC                    Unknown   
4            15    3.0    PA                       True   

  RestaurantsPriceRange2 RestaurantsTakeOut  Parking  
0                      2               True    False  
1                      1               True    False  
2                      2               True    False  
3                Unknown            Unknown    False  
4                      1               True     True

# Export Pre-Processed Data to Fit Models in R 
We initially wanted to create decision trees and random forests in python, but it would not deal with categorical data well. We would have to convert all the data to numerics and it would be hard to place a ordering on the numerics. Thus, we are exporting the dataset, because R can create the models more easily and ones that make more sense. 

In [97]:
restaurantDf.to_csv("restaurantInfo.csv") 